In [1]:
from stata_preprocessing import *
from llm_simulation import *
from load_environmental_distributions import load_environmental_distribution
# from visualise_responses import *
from fine_tuning_prompts import *
import os
import random
from openai import OpenAI

In [2]:
stata_data_filepath = "C:\\Users\\haoch\\Documents\\COMP0190\\Data\\COMP0191-MSc-Project-Code\\Stata-Results\\UKHLS_fine_tuning_responses.dta"
base_environmental_views_filepath = "C:\\Users\\haoch\\Documents\\COMP0190\\Data\\COMP0191-MSc-Project-Code\\Environmental-Views-Variables\\"

In [3]:
stata_data = load_stata_file(stata_data_filepath)
stata_response_wave_eight, stata_response_wave_nine, stata_response_wave_ten = [stata_data[stata_data["wavename"] == wavename] for wavename in [8.0, 9.0, 10.0]]

In [4]:
print(stata_response_wave_ten.head())

          pidp       hidp fihhmnlabgrs_dv     sex                 marstat  \
109302   22445  277059218     5996.049805  female                 Married   
109303   29925  618630018          1666.0  female                Divorced   
109304   76165  141460418          6732.0  female                 Married   
109305  280165  754371618         12333.0  female                 Married   
109306  333205  414800018     4782.819824  female  Single, nvr marr/civ p   

              qfhigh                                           racel_dv  \
109302  inapplicable  british/english/scottish/welsh/northern irish ...   
109303  inapplicable  british/english/scottish/welsh/northern irish ...   
109304  inapplicable  british/english/scottish/welsh/northern irish ...   
109305  inapplicable  british/english/scottish/welsh/northern irish ...   
109306  inapplicable  british/english/scottish/welsh/northern irish ...   

              lnprnt age_dv         gor_dv  ...       agegr10_dv  \
109302  inapplicab

In [5]:
stata_response_wave_ten = preprocess_stata_data(stata_response_wave_ten)